1. Importing Tools & Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from imblearn.over_sampling import SMOTE
import joblib
import os

2. Load Dataset

In [2]:
df=pd.read_csv("creditcard.csv")

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df=df.apply(lambda x : x.astype(int) if x.dtype=='float64'else x)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,149,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,1,-1,-1,1,0,0,1,0,0,-1,...,0,0,0,0,0,0,0,0,378,0
3,1,0,0,1,0,0,1,0,0,-1,...,0,0,0,-1,0,0,0,0,123,0
4,2,-1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,69,0


3. Preprocess Data

In [5]:
def preprocess_data(df):
    x = df.drop("Class",axis=1)
    y = df["Class"]
    # Apply SMOTE to handle class imbalance
    smote = SMOTE(random_state=42)
    x_resampled,y_resampled=smote.fit_resample(x,y)
    return train_test_split(x_resampled,y_resampled,test_size=0.2,random_state=42)

4. Train Model

In [6]:
x = df.drop("Class",axis=1)
y = df["Class"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)


In [7]:
def train_model(x_train,y_train):
    model = RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(x_train,y_train)
    return model

5. Evaluate Model

In [8]:
def evalute_model(model,x_test,y_test):
    y_pred = model.predict(x_test)
    print("\n Model Evaluation Report :\n")
    print(classification_report(y_test,y_pred))

In [9]:
print("\n Model Evaluation Report :\n")
print(classification_report(y_test,y_pred))


 Model Evaluation Report :

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.80      0.88        98

    accuracy                           1.00     56962
   macro avg       0.99      0.90      0.94     56962
weighted avg       1.00      1.00      1.00     56962



6. Save the Model

In [11]:
def save_model(filename='fraud_detection_model.pkl'):
    joblib.dump(model,filename)

7. Load the Model

In [13]:
def load_model(filename='fraud_detection_model.pkl'):
    return joblib.load(filename)

8. Command Line Interface For Testing

In [15]:
def test_interface(model):
    print("\n----Fraud Detection Testing Interface----")
    print("Enter 30 features as comma-seperated values:")
    try:
        user_input=input(">> ")
        features=np.array([float(x.strip()) for x in user_input.split(",")])
        if len(features) != 30:
            print("Error : please provide exactly 30 feature values.")
            return
        prediction=model.predict([features])[0]
        print("Result : Fradulent Transaction" if prediction == 1 else "Result : Legitimate Transaction")
    except Exception as e :
        print(f"Invalid input or error occur :{e}")

In [17]:
x_train,x_test,y_train,y_test=preprocess_data(df)

In [ ]:
model=train_model(x_train,y_train)